# Animal profile

This file creates the profile output for a group of animals.

| ![](../images/profile.jpg) | 
|:--:| 
|Example of profile. 1 point per animal. Each color represents one cage|

This code computes the total time spent in each behaviour as well as the total number of occurrences of this behaviour for each individual. You can choose to compute this profile either over all the experiment or for each night separately. It provides a plot for the total duration of all the different events and a plot for the total number of occurrences of each behaviour as pdf files. It also provides a txt file with all individual values for each variable, as well as statistics (mixed model for each trait, uncorrected: trait as a function of genotype, with a group effect).


In [3]:
'''
Created on 13 sept. 2017

@author: Fabrice de Chaumont and Elodie Ey
'''

import sys
sys.path.insert(1, "../")

import sqlite3
from lmtanalysis.Animal import *
import numpy as np
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
import colorsys
from collections import Counter
import seaborn as sns

from tkinter.filedialog import askopenfilename
from lmtanalysis.Util import getMinTMaxTAndFileNameInput
from lmtanalysis.EventTimeLineCache import EventTimeLineCached
from lmtanalysis.FileUtil import getFilesToProcess
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas
from datetime import datetime


def computeProfile(file, minT, maxT, night):
    
    connection = sqlite3.connect( file )
    
    pool = AnimalPool()
    pool.loadAnimals( connection )
    
    animalData = {} #will then be the 'profileData'
    # animalData[night] = night #Assigning the night

    for animal in pool.animalDictionnary.keys():
        print(f"computing individual animal:{animal}")
        rfid = pool.animalDictionnary[animal].RFID
        print(f"RFID:{rfid}")
        animalData[rfid]= {}        
        #store the animal
        animalData[rfid]["animal"] = pool.animalDictionnary[animal]
        animalData[rfid]["file"] = file
        
        genoA = None
        try:
            genoA=pool.animalDictionnary[animal].genotype
        except:
            pass
                    
        for behavEvent in behavioralEvents[:-2]:
            
            print(f"computing individual event: {behavEvent}")    
            
            behavEventTimeLine = EventTimeLineCached( connection, file, behavEvent, animal, minFrame=minT, maxFrame=maxT )
            
            totalEventDuration = behavEventTimeLine.getTotalLength()
            nbEvent = behavEventTimeLine.getNumberOfEvent(minFrame = minT, maxFrame = maxT )
            print( "total event duration: " , totalEventDuration )                
            animalData[rfid][behavEventTimeLine.eventName+" TotalLen"] = totalEventDuration
            animalData[rfid][behavEventTimeLine.eventName+" Nb"] = nbEvent
            
            print(behavEventTimeLine.eventName, genoA, behavEventTimeLine.idA, totalEventDuration, nbEvent)

    header = ["file","strain","sex","group","day","exp","RFID","genotype", "user1", "minTime","maxTime"]
    for name in header:
        text_file.write( "{}\t".format ( name ) ) 
    for kAnimal in animalData:    
        #identify the experiment where the animal comes from:
        animalData[kAnimal]["experiment"] = file  #HD: What does it do ?
        #compute the total distance traveled
        COMPUTE_TOTAL_DISTANCE = True #Change to 'True' OR 'False' if you want OR not to compute total Distance
        if COMPUTE_TOTAL_DISTANCE == True:
            animalData[kAnimal]["animal"].loadDetection( start=minT, end=maxT, lightLoad = True )
            animalData[kAnimal]["totalDistance"] = animalData[kAnimal]["animal"].getDistance( tmin=minT,tmax=maxT)/100
        else:
            animalData[kAnimal]["totalDistance"] = "totalDistance"
        
    #write event keys
    firstAnimalKey = next(iter(animalData))
    firstAnimal = animalData[firstAnimalKey]
    for k in firstAnimal.keys():
        text_file.write( "{}\t".format( k.replace(" ", "") ) )
    text_file.write("\n")
    
    for kAnimal in animalData:
        text_file.write( "{}\t".format( file ) )
        text_file.write( "{}\t".format( "strain" ) )
        text_file.write( "{}\t".format( "sex" ) )
        text_file.write( "{}\t".format( "group" ) )
        text_file.write( "{}\t".format( f"night-{night}" ) )
        text_file.write( "{}\t".format( "exp" ) )
        text_file.write( "{}\t".format( animalData[kAnimal]["animal"].RFID ) )
        text_file.write( "{}\t".format( animalData[kAnimal]["animal"].genotype ) )
        text_file.write( "{}\t".format( animalData[kAnimal]["animal"].user1 ) )
        text_file.write( "{}\t".format( minT ) )
        text_file.write( "{}\t".format( maxT ) )

        for kEvent in firstAnimal.keys():
            text_file.write( "{}\t".format( animalData[kAnimal][kEvent] ) )
        text_file.write( "\n" )
        
    connection.close() #Close the connection to the database (to save memory?)
    return animalData


def getProfileValues(profileData, night=0, event=None, nbNightsDic=None):

#     print(f"in getProfileValues, night is = {night}")
    dataDic = {}
    dataDic["genotype"] = []
    dataDic["value"] = []
    dataDic["exp"] = []
    for file in profileData.keys():
        if night == 0 or night <= nbNightsDic[file]: #Check if there are more nights in the file
            for animal in profileData[file][night]: # BUG WITH night=1 ?
                if (genoToRemove.lower() != "no") & (genoToRemove != ""): #Test that it is not "NO" or ""
                    if genoToRemove in profileData[file][night][animal]["animal"].genotype:  #SKIPS THE 'genoToRemove' ANIMALS !!!
                        continue
                dataDic["value"].append(profileData[file][night][animal][event])
                dataDic["exp"].append(profileData[file][night][animal]["experiment"])
                dataDic["genotype"].append(profileData[file][night][animal]["animal"].genotype)
        else:
            print(f"During night#{night}, file '{file}' EXLUDED !!!")
            continue
    return dataDic


def plotProfileDataDuration(profileData, night, valueCat, nbNightsDic=None):
    fig, axes = plt.subplots(nrows=5, ncols=6, figsize=(14, 12))
#     plt.xticks(rotation=45, horizontalalignment='right', fontweight='light') #X-axis 45° angle

    row=0
    col=0
    fig.suptitle(t=f"{valueCat} of events (night {night})", y=1.2, fontweight= 'bold')
    
    #plot the data for each behavioural event
    for behavEvent in behavioralEvents[:-2]: # HD: WHY '-2' ??? Not taking the Last Behavior (totalDist) ?
        event = behavEvent+valueCat
        print("event: ", event)

        profileValueDictionary = getProfileValues(profileData=profileData, night=night,
                                                  event=event, nbNightsDic=nbNightsDic)
        y = profileValueDictionary["value"]
        x = profileValueDictionary["genotype"]
        genotypeType = Counter(x)
        group = profileValueDictionary["exp"]
        
        print("y: ", y)
        print("x: ", x)
        print("group: ", group)
        experimentType = Counter(group)
        print("Nb of experiments: ", len(experimentType))
        
        axes[row,col].set_xlim(-0.5, 1.5)
        axes[row,col].set_ylim(min(y)-0.2*max(y), max(y)+0.2*max(y))
        sns.stripplot(x, y, jitter=True, hue=group, s=5, ax=axes[row,col] )
        axes[row,col].set_title(behavEvent)
        axes[row,col].set_ylabel("{} (frames)".format(valueCat))
        axes[row,col].legend().set_visible(False)
        axes[row,col].spines['right'].set_visible(False)
        axes[row,col].spines['top'].set_visible(False)
                
        if col<5:
            col+=1
            row=row
        else:
            col=0
            row+=1

    #plot the data for the total distance traveled   
    print("plot Total Distance")
    profileValueDictionary = getProfileValues(profileData=profileData, night=night, 
                                              event="totalDistance", nbNightsDic=nbNightsDic)
    y = profileValueDictionary["value"]
    x = profileValueDictionary["genotype"]
    genotypeType = Counter(x)
    group = profileValueDictionary["exp"]
    
    print("y: ", y)
    print("x: ", x)
    print("group: ", group)
    experimentType = Counter(group)
    print("Nb of experiments: ", len(experimentType))
    
    axes[row,col].set_xlim(-0.5, 1.5)
    axes[row,col].set_ylim(min(y)-0.2*max(y), max(y)+0.2*max(y))
    sns.stripplot(x, y, jitter=True, hue=group, s=5, ax=axes[row,col] )
    axes[row,col].set_title("Activity")
    axes[row,col].set_ylabel("total distance (m)")
    axes[row,col].legend().set_visible(False)
    axes[row,col].spines['right'].set_visible(False)
    axes[row,col].spines['top'].set_visible(False)
    
    if col<7:
        col+=1
        row=row
    else:
        col=0
        row+=1
    
    
    #ROTATE THE X-AXIS OF ALL SUBPLOTS
    for ax in fig.axes:
        matplotlib.pyplot.sca(ax)
        plt.xticks(rotation=45)
        
    fig.tight_layout()
    figFileName = f"{text_file_name}_{valueCat}_night_{night}.png"
    print("Saving " , figFileName )
    fig.savefig( figFileName ,dpi=100)
    plt.close( fig )
    

def testProfileData(profileData=None, night=0, eventListNames=None, valueCat="", 
                    text_file=None, nbNightsDic=None):
    for behavEvent in eventListNames:
        event = behavEvent+valueCat
        print("event: ", event)
        text_file.write("Test for the event: {} night {}".format( event, night ) )
        
        profileValueDictionary = getProfileValues(profileData=profileData, night=night, 
                                                  event=event, nbNightsDic=nbNightsDic)
        
        dfData = pandas.DataFrame({'group': profileValueDictionary["exp"],
                                   'genotype': profileValueDictionary["genotype"],
                                   'value': profileValueDictionary["value"]})
        
        #pandas.DataFrame(dfData).info()
        
        #Mixed model: variable to explain: value; fixed factor = genotype; random effect: group
        model = smf.mixedlm("value ~ genotype", dfData, groups = dfData["group"])  #Creates the model 
        result = model.fit()  #Run model
        print(result.summary())  #Print summary
        text_file.write(result.summary().as_text())


if __name__ == '__main__':
    
    print("Code launched.")

    # datetime object containing current date and time
    now = datetime.now()
    print("Start @", now)
    
    files = getFilesToProcess()
    tmin, tmax, text_file, text_file_name = getMinTMaxTAndFileNameInput()

    profileData = {}
    M_profileData = {}
    F_profileData = {}
    
    nightComputation = input("Compute profile only during night events (Yes or No)? ")
    doStats = input("Do you want to do Stats on your data (Yes or No)? (Warning: MORE THAN ONE FILE REQUIRED): ")
    genoToRemove = ""
    while genoToRemove == "":
        genoToRemove = input("Is there a Genotype to exclude? (a 'keyword' or 'NO' if None) DON'T LEAVE BLANK!): ")
    
    splitBySex_input = input("Do you want to split by Sex (Yes or No)? (creates differnt graph and stats for M and F")
    if splitBySex_input.lower() == "no":
        print(" => Data won't be split for both Sex")
        splitBySex = False
    elif splitBySex_input.lower() == "yes":
        print(" => Data will be split for both Sex")
        splitBySex = True
        
        M_text_file_name = "M_"+text_file_name
        M_text_file = open(M_text_file_name, "w")
        F_text_file_name = "F_"+text_file_name
        F_text_file = open(F_text_file_name, "w")
    else:
        print(" => No clear answer... data won't be split by Sex!")
        splitBySex = False
        
    nbMaxAnimal = int(input("What's the MAX number of Animals per LMT ? (1,2,3 or 4): "))
    
    ### List of events to be computed within the behavioral profile     ###
    ### and header for the computation of the total distance travelled. ###
    
    if nbMaxAnimal == 2:
        behavioralEvents = ["Contact", "Oral-oral Contact", "Oral-genital Contact", "Side by side Contact", 
                            "Side by side Contact, opposite way", "Social approach", "Get away", "Break contact", 
                            "Approach contact","Approach rear", "FollowZone Isolated", "Train2", "Group2", 
                            "Move isolated", "Move in contact", "Rear isolated", "Rear in contact", "Stop isolated", 
                            "WallJump","Water Zone","Fight","Gets to Fight","Won Fight","Lost Fight", "SAP",
                            "extra", "totalDistance"]
    if nbMaxAnimal == 3:
        behavioralEvents = ["Contact", "Oral-oral Contact", "Oral-genital Contact", "Side by side Contact", 
                            "Side by side Contact, opposite way", "Social approach", "Get away", "Approach contact", 
                            "Approach rear", "Break contact", "FollowZone Isolated", "Train2", "Group2", "Group3",
                            "Group 3 break", "Group 3 make", "Move isolated", "Move in contact", "Rear isolated", "SAP",
                            "Rear in contact", "Stop isolated", "WallJump","extra", "totalDistance"]
    if nbMaxAnimal == 4:
        behavioralEvents = ["Contact", "Oral-oral Contact", "Oral-genital Contact", "Side by side Contact", 
                            "Side by side Contact, opposite way", "Social approach", "Get away", "Approach contact", 
                            "Approach rear", "Break contact", "FollowZone Isolated", "Train2", "Group2", "Group3", "Group4",
                            "Group 3 break", "Group 3 make", "Group 4 break", "Group 4 make", "Move isolated", "SAP",
                            "Move in contact", "Rear isolated", "Rear in contact", "Stop isolated", 
                            "WallJump","extra", "totalDistance"]
    
#     behavioralEvents = ["Contact", "Oral-oral Contact", "Oral-genital Contact", "Side by side Contact", 
#                         "Side by side Contact, opposite way", "Social approach", "Get away", "Approach contact", 
#                         "Approach rear", "Break contact", "FollowZone Isolated", "Train2", "Group2", "Group3", "Group4",
#                         "Group 3 break", "Group 3 make", "Group 4 break", "Group 4 make", "Move isolated", 
#                         "Move in contact", "Rear isolated", "Rear in contact", "Stop isolated", "SAP",
#                         "WallJump","extra", "totalDistance"]
#     behavioralEvents = ["Fight","Gets to Fight","Won Fight","Lost Fight","extra", "totalDistance"]

    for file in files:
        print(file)
        connection = sqlite3.connect( file )
        profileData[file] = {}
        M_profileData[file] = {}
        F_profileData[file] = {}
        
        pool = AnimalPool()
        pool.loadAnimals(connection)
        
        if splitBySex == True:
            print(f"There are {pool.getNbAnimals()} mice in this cage.")
            Males = 0
            Females = 0
            animalList = pool.getAnimalList()
            for animal in animalList:
                if animal.sex == "M":
                    Males += 1
                    print(f" - animal with RFID{animal.RFID} is a Male")
                if animal.sex == "F":
                    Females += 1
                    print(f" - animal with RFID{animal.RFID} is a Female")
                else:
                    print(f"the Sex is defined as *{animal.sex}*")

            if Males == pool.getNbAnimals():
                print("All mice in this cage are Males")
                Sex = "M"
                
                if nightComputation.lower() == "no":
        #             print("no-1")
                    minT = tmin
                    maxT = tmax
                    n = 0
                    #Compute profile data and save them in a text file
                    M_profileData[file][n] = computeProfile(file = file, minT=minT, maxT=maxT, night=n)
                    M_text_file.write( "\n" )
                    print("M Profile data saved.")

                if nightComputation.lower() == "yes":
        #             print("yes-1")
                    nightEventTimeLine = EventTimeLineCached(connection, file, "night", minFrame=tmin, maxFrame=tmax )
                    n = 1

                    for eventNight in nightEventTimeLine.getEventList():
                        minT = eventNight.startFrame
                        maxT = eventNight.endFrame
                        print("* * **   Night: ", n, "   ** * *")
                        print(minT, maxT)
                        #Compute profile data and save them in a text file
                        M_profileData[file][n] = computeProfile(file=file, minT=minT, maxT=maxT, night=n)
                        M_text_file.write( "\n" )
                        print("-- M Profile data saved. --")

                        n+=1 #increase 'n' for the following nights
                else: #Neither 'yes' or 'no' for night computation
                    print("There is a problem with your 'compute night' answer!")
            if Females == pool.getNbAnimals():
                print("All mice in this cage are Females")
                Sex = "F"
                
                if nightComputation.lower() == "no":
        #             print("no-1")
                    minT = tmin
                    maxT = tmax
                    n = 0
                    #Compute profile data and save them in a text file
                    F_profileData[file][n] = computeProfile(file = file, minT=minT, maxT=maxT, night=n)
                    F_text_file.write( "\n" )
                    print("M Profile data saved.")

                if nightComputation.lower() == "yes":
        #             print("yes-1")
                    nightEventTimeLine = EventTimeLineCached(connection, file, "night", minFrame=tmin, maxFrame=tmax )
                    n = 1

                    for eventNight in nightEventTimeLine.getEventList():
                        minT = eventNight.startFrame
                        maxT = eventNight.endFrame
                        print("* * **   Night: ", n, "   ** * *")
                        print(minT, maxT)
                        #Compute profile data and save them in a text file
                        F_profileData[file][n] = computeProfile(file=file, minT=minT, maxT=maxT, night=n)
                        F_text_file.write( "\n" )
                        print("-- M Profile data saved. --")

                        n+=1 #increase 'n' for the following nights
                        
                else: #Neither 'yes' or 'no' for night computation
                    print("There is a problem with your 'compute night' answer!")
                
            else:
                print("There is a mix of M and F in this cage ?!")
                Sex = "mixed"
            
#             profileData = M_profileData + F_profileData
            profileData = dict(M_profileData)
            profileData.update(F_profileData)
                
################################################
        
        if splitBySex == False:
            if nightComputation.lower() == "no":
    #             print("no-1")
                minT = tmin
                maxT = tmax
                n = 0
                #Compute profile data and save them in a text file
                profileData[file][n] = computeProfile(file = file, minT=minT, maxT=maxT, night=n)
                text_file.write( "\n" )
                print("Profile data saved.")

            if nightComputation.lower() == "yes":
    #             print("yes-1")
                nightEventTimeLine = EventTimeLineCached(connection, file, "night", minFrame=tmin, maxFrame=tmax )
                n = 1

                for eventNight in nightEventTimeLine.getEventList():
                    minT = eventNight.startFrame
                    maxT = eventNight.endFrame
                    print("* * **   Night: ", n, "   ** * *")
                    print(minT, maxT)
                    #Compute profile data and save them in a text file
                    profileData[file][n] = computeProfile(file=file, minT=minT, maxT=maxT, night=n)
                    text_file.write( "\n" )
                    print("-- Profile data saved. --")

                    n+=1 #increase 'n' for the following nights
            else: #Neither 'yes' or 'no' for night computation
                print("There is a problem with your 'compute night' answer!")
            
        connection.close() #Close the connection with database (Saves memory?)

    text_file.write( "\n" )
    print("--- Write text before plotting ProfileData ---")    
    
    
    if splitBySex == False:
        print("### Checking number of nights (Data not Split by 'Sex') ###")
        nb_nights = {}
        for file in profileData:
            print(file)
            nb_nights[file]=len(profileData[file])
            print(len(profileData[file]))
        print(nb_nights)
        nb_max_nights = max(nb_nights.keys(), key=nb_nights.get)
        nb_min_nights = min(nb_nights.keys(), key=nb_nights.get)

        print(f'Maximum number of nights: {nb_nights[nb_max_nights]} in {nb_max_nights}')
        print(f'Minimum number of nights: {nb_nights[nb_min_nights]} in {nb_min_nights}')
    
        if nightComputation.lower() == "no":
    #         print("no-2")
            n = 0

            #Plot profile data and save them in a png figure
            plotProfileDataDuration(profileData=profileData, night=n, valueCat=" TotalLen", nbNightsDic=None)
            plotProfileDataDuration(profileData=profileData, night=n, valueCat=" Nb", nbNightsDic=None)

            #Test profile data and save results in a text file
            text_file.write( "Statistical analysis: mixed linear models" )
            text_file.write( "{}\n" )
            testProfileData(profileData=profileData, night=n, eventListNames=behavioralEvents[:-2], 
                            valueCat=" TotalLen", text_file=text_file, nbNightsDic=None)
            testProfileData(profileData=profileData, night=n, eventListNames=behavioralEvents[:-2], 
                            valueCat=" Nb", text_file=text_file, nbNightsDic=None)
            print("test for total distance")
            testProfileData(profileData=profileData, night=n, eventListNames=["totalDistance"], 
                            valueCat="", text_file=text_file, nbNightsDic=None)

        if nightComputation.lower() == "yes":
            #Following line removed because 'nightEventTimeLine' is already computed before
    #         nightEventTimeLine = EventTimeLineCached( connection, file, "night", minFrame=tmin, maxFrame=tmax )

            n = 1 #night counter

            for eventNight in nightEventTimeLine.getEventList():
                print("* * **  *** Night: ", n, " ***  ** * *")

                #Plot profile data and save them
                plotProfileDataDuration(profileData=profileData, night=n, valueCat=" TotalLen", nbNightsDic=nb_nights)
                plotProfileDataDuration(profileData=profileData, night=n, valueCat=" Nb", nbNightsDic=nb_nights)


    #             plotProfileDataDuration(profileData=profileData, night=n, valueCat=" TotalLen")
    #             plotProfileDataDuration(profileData=profileData, night=n, valueCat=" Nb")

                if doStats.lower() == "yes" :
                    #Test profile data and save results in a text file
                    print(" -- Now, doing some stats...")
                    text_file.write( "Statistical analysis: mixed linear models" )
                    text_file.write( "{}\n" )
                    testProfileData(profileData=profileData, night=n, eventListNames=behavioralEvents[:-2], 
                                    valueCat=" TotalLen", text_file=text_file, nbNightsDic=nb_nights)
                    testProfileData(profileData=profileData, night=n, eventListNames=behavioralEvents[:-2], 
                                    valueCat=" Nb", text_file=text_file, nbNightsDic=nb_nights)
                    print("test for total distance")
                    testProfileData(profileData=profileData, night=n, eventListNames=["totalDistance"], 
                                    valueCat="", text_file=text_file, nbNightsDic=nb_nights)
                n+=1 #Implement the night counter
    #             if n > nb_min_nights:
    #                 print("Different number of nights in the databases provided, the computation stops here ...")

    if splitBySex == True:
        
        print("### Checking number of nights for Males ###")
        M_nb_nights = {}
        for M_file in M_profileData:
            print(M_file)
            M_nb_nights[M_file]=len(M_profileData[M_file])
            print(len(M_profileData[M_file]))
        print(M_nb_nights)
        M_nb_max_nights = max(M_nb_nights.keys(), key=M_nb_nights.get)
        M_nb_min_nights = min(M_nb_nights.keys(), key=M_nb_nights.get)

        print(f'Maximum number of nights: {M_nb_nights[M_nb_max_nights]} in {M_nb_max_nights}')
        print(f'Minimum number of nights: {M_nb_nights[M_nb_min_nights]} in {M_nb_min_nights}')
              
        print("### Checking number of nights for Females ###")
        F_nb_nights = {}
        for F_file in F_profileData:
            print(F_file)
            F_nb_nights[F_file]=len(F_profileData[F_file])
            print(len(F_profileData[F_file]))
        print(F_nb_nights)
        F_nb_max_nights = max(F_nb_nights.keys(), key=F_nb_nights.get)
        F_nb_min_nights = min(F_nb_nights.keys(), key=F_nb_nights.get)

        print(f'Maximum number of nights: {F_nb_nights[F_nb_max_nights]} in {F_nb_max_nights}')
        print(f'Minimum number of nights: {F_nb_nights[F_nb_min_nights]} in {F_nb_min_nights}')
              
        if nightComputation.lower() == "no":
#         print("no-2")
            n = 0

            ## # FOR MALES # ##
            #Plot profile data and save them in a png figure
            M_plotProfileDataDuration(profileData=M_profileData, night=n, valueCat=" TotalLen", nbNightsDic=None)
            M_plotProfileDataDuration(profileData=M_profileData, night=n, valueCat=" Nb", nbNightsDic=None)

            #Test profile data and save results in a text file
            M_text_file.write( "Statistical analysis: mixed linear models" )
            M_text_file.write( "{}\n" )
            
            testProfileData(profileData=M_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                            valueCat=" TotalLen", text_file=M_text_file, nbNightsDic=None)
            testProfileData(profileData=M_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                            valueCat=" Nb", text_file=M_text_file, nbNightsDic=None)
            print("test for total distance")
            testProfileData(profileData=M_profileData, night=n, eventListNames=["totalDistance"], 
                            valueCat="", text_file=M_text_file, nbNightsDic=None)
            
            ## # FOR FEMALES # ##
            #Plot profile data and save them in a png figure
            F_plotProfileDataDuration(profileData=F_profileData, night=n, valueCat=" TotalLen", nbNightsDic=None)
            F_plotProfileDataDuration(profileData=F_profileData, night=n, valueCat=" Nb", nbNightsDic=None)

            #Test profile data and save results in a text file
            F_text_file.write( "Statistical analysis: mixed linear models" )
            F_text_file.write( "{}\n" )
            
            testProfileData(profileData=F_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                            valueCat=" TotalLen", text_file=F_text_file, nbNightsDic=None)
            testProfileData(profileData=F_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                            valueCat=" Nb", text_file=F_text_file, nbNightsDic=None)
            print("test for total distance")
            testProfileData(profileData=F_profileData, night=n, eventListNames=["totalDistance"], 
                            valueCat="", text_file=F_text_file, nbNightsDic=None)

        if nightComputation.lower() == "yes":
            #Following line removed because 'nightEventTimeLine' is already computed before
    #         nightEventTimeLine = EventTimeLineCached( connection, file, "night", minFrame=tmin, maxFrame=tmax )

            n = 1 #night counter

            for eventNight in nightEventTimeLine.getEventList():
                print("* * **  *** Night: ", n, " ***  ** * *")

                ## # FOR MALES # ##
                #Plot profile data and save them
                plotProfileDataDuration(profileData=M_profileData, night=n, 
                                        valueCat=" TotalLen", nbNightsDic=M_nb_nights)
                plotProfileDataDuration(profileData=M_profileData, night=n, 
                                        valueCat=" Nb", nbNightsDic=M_nb_nights)
                
                ## # FOR FEMALES # ##
                #Plot profile data and save them
                plotProfileDataDuration(profileData=F_profileData, night=n, 
                                        valueCat=" TotalLen", nbNightsDic = F_nb_nights)
                plotProfileDataDuration(profileData=F_profileData, night=n, 
                                        valueCat=" Nb", nbNightsDic = F_nb_nights)

                if doStats.lower() == "yes" :
                    #Test profile data and save results in a text file
                    
                    ## # FOR MALES # ##
                    print(" -- Now, doing some stats... For males...")
                    M_text_file.write( "Statistical analysis: mixed linear models" )
                    M_text_file.write( "{}\n" )
                    testProfileData(profileData=M_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                                    valueCat=" TotalLen", text_file=M_text_file, nbNightsDic=M_nb_nights)
                    testProfileData(profileData=M_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                                    valueCat=" Nb", text_file=M_text_file, nbNightsDic=M_nb_nights)
                    print("test for total distance")
                    testProfileData(profileData=M_profileData, night=n, eventListNames=["totalDistance"], 
                                    valueCat="", text_file=M_text_file, nbNightsDic=M_nb_nights)
                    
                    ## # FOR FEMALES # ##
                    print(" -- Now, doing some stats... For Females...")
                    F_text_file.write( "Statistical analysis: mixed linear models" )
                    F_text_file.write( "{}\n" )
                    testProfileData(profileData=F_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                                    valueCat=" TotalLen", text_file=F_text_file, nbNightsDic=F_nb_nights)
                    testProfileData(profileData=F_profileData, night=n, eventListNames=behavioralEvents[:-2], 
                                    valueCat=" Nb", text_file=F_text_file, nbNightsDic=F_nb_nights)
                    print("test for total distance")
                    testProfileData(profileData=F_profileData, night=n, eventListNames=["totalDistance"], 
                                    valueCat="", text_file=F_text_file, nbNightsDic=F_nb_nights)
                    
                n+=1 #Implement the night counter
    #             if n > nb_min_nights:
    #                 print("Different number of nights in the databases provided, the computation stops here ...")

    print("") 
    print( "***************************************************")
    print ("Plots saved as png and analyses saved in text file.")
    text_file.close()
    M_text_file.close()
    F_text_file.close()

    print ("*** ALL JOBS DONE ***")
    # datetime object containing current date and time
    now = datetime.now()
    print("Ends @", now)
    

Code launched.
Start @ 2020-06-15 13:23:41.168926
Folder:  C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1
Fetching files...
C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200227_C2-triads_Cage1_LMT1 - Copy.sqlite found.
C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200229_C2-triads_Cage2_LMT1 - Copy.sqlite found.
C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200302_C2-triads_Cage3_LMT1 - Copy.sqlite found.
C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200304_C2-triads_Cage4_LMT1 - Copy.sqlite found.
C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200310_C2-triads_Cage5_LMT1 - Copie.sqlite found.
C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200425_C2-triads_Cage6_LMT1 - Copy.sqlite found.
C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200427_C2-triads_Cage7_LMT1 - Copy.sqlite found.
Enter time information in frame. You can also set in days, hour, minutes
valid entries are: 100, 1d, 1.5d, 23.5h, 1d 2h 3m 4s 5f
Starting t : 0
Entry (in frame) : 0
Ending t : 
Enter file name to save da

C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anacond

                 Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    value         
No. Observations:      7          Method:                REML          
No. Groups:            3          Scale:                 393754299.0106
Min. group size:       2          Likelihood:            -57.7242      
Max. group size:       3          Converged:             No            
Mean group size:       2.3                                             
-----------------------------------------------------------------------
                  Coef.     Std.Err.   z    P>|z|   [0.025     0.975]  
-----------------------------------------------------------------------
Intercept       359354.808 13515.306 26.589 0.000 332865.295 385844.322
genotype[T.-/-] -38183.055 15151.168 -2.520 0.012 -67878.798  -8487.312
Group Var        16575.559                                             

event:  Oral-oral Contact TotalLen
During night#1, file 'C:/C2 triads/DB Copys, 

C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anac

                 Mixed Linear Model Regression Results
Model:                MixedLM     Dependent Variable:     value        
No. Observations:     7           Method:                 REML         
No. Groups:           3           Scale:                  38271033.0388
Min. group size:      2           Likelihood:             -51.9513     
Max. group size:      3           Converged:              No           
Mean group size:      2.3                                              
-----------------------------------------------------------------------
                   Coef.    Std.Err.   z    P>|z|   [0.025     0.975]  
-----------------------------------------------------------------------
Intercept        121787.836 5755.570 21.160 0.000 110507.127 133068.545
genotype[T.-/-]  -12810.105 8419.895 -1.521 0.128 -29312.796   3692.587
Group Var       1135310.543 7484.670                                   

event:  Approach rear TotalLen
During night#1, file 'C:/C2 triads/DB Copys, Rebu

...

               Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    value        
No. Observations:    9          Method:                REML         
No. Groups:          3          Scale:                 16740606.6821
Min. group size:     3          Likelihood:            -69.6489     
Max. group size:     3          Converged:             No           
Mean group size:     3.0                                            
--------------------------------------------------------------------
                  Coef.   Std.Err.   z    P>|z|   [0.025     0.975] 
--------------------------------------------------------------------
Intercept       31551.446 1761.804 17.909 0.000  28098.373 35004.519
genotype[T.-/-] -6827.003 2505.979 -2.724 0.006 -11738.633 -1915.374
Group Var       10219.004                                           

event:  FollowZone Isolated TotalLen
During night#2, file 'C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200227_C2-tr

C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anacond

               Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    value       
No. Observations:    9          Method:                REML        
No. Groups:          3          Scale:                 3517738.4434
Min. group size:     3          Likelihood:            -64.2560    
Max. group size:     3          Converged:             No          
Mean group size:     3.0                                           
-------------------------------------------------------------------
                  Coef.   Std.Err.   z    P>|z|   [0.025    0.975] 
-------------------------------------------------------------------
Intercept       14937.906  857.617 17.418 0.000 13257.008 16618.803
genotype[T.-/-]   331.462 1263.371  0.262 0.793 -2144.700  2807.624
Group Var       88098.070  897.795                                 

event:  Move in contact Nb
During night#2, file 'C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200227_C2-triads_Cage1_LMT1 - Copy

C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Damien\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)



event:  Rear in contact Nb
During night#2, file 'C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200227_C2-triads_Cage1_LMT1 - Copy.sqlite' EXLUDED !!!
During night#2, file 'C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200304_C2-triads_Cage4_LMT1 - Copy.sqlite' EXLUDED !!!
During night#2, file 'C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200310_C2-triads_Cage5_LMT1 - Copie.sqlite' EXLUDED !!!
During night#2, file 'C:/C2 triads/DB Copys, Rebuilt, Corrected/LMT1\200425_C2-triads_Cage6_LMT1 - Copy.sqlite' EXLUDED !!!
              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    value      
No. Observations:    9          Method:                REML       
No. Groups:          3          Scale:                 180247.0380
Min. group size:     3          Likelihood:            -56.1267   
Max. group size:     3          Converged:             Yes        
Mean group size:     3.0                                          
---------------------

In [7]:
text_file

<_io.TextIOWrapper name='test.txt' mode='w' encoding='cp1252'>